In [1]:
import pandas as pd
import numpy as np
import sys
import datetime
from datetime import date
import os, fnmatch
import numpy as np
_path=('/home/ak/Documents/Data/GPDATA/')
data_path= ('/home/ak/Documents/Data/')
import shutil
### make a list of all the files, this will be used to parse through ###
### ignore files that are there before ###
_files=fnmatch.filter(os.listdir(_path), 'garethpet*.csv')
sys.path.append(_path)

no_files=len(_files)

In [2]:
###define functions which will be used###

def _raw_data(file_name):
    raw_data =pd.read_csv(file_name, low_memory=False)
    return raw_data

def _columns(data_frame):
    _columns_ =data_frame.columns.values
    return _columns_

def _trade_data(data_frame):
    trade_data = data_frame[data_frame['Type'] == 'Trade']
    return trade_data


def _basic_columns(data_frame):
    basic_columns=data_frame[['#RIC','Date[G]','Time[G]','Price','Volume']]
    return basic_columns

def _date_convert(_date):
    return datetime.date.strftime(datetime.datetime.strptime(_date, '%d-%b-%Y'), '%Y%m%d')

def time_in_millis(time_stamp):
    conv_time =datetime.datetime.strptime(time_stamp, '%H:%M:%S.%f')
    in_millis=(((conv_time.hour.real*60 + conv_time.minute.real) *60)
               +(conv_time.second.real))*1000 +float(conv_time.microsecond)/10**3.
    return in_millis

def listdirs(folder): #return only directories from a master folder
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]

def copy_directory(dir_src, dir_dst ): #copy all files from one directory with one extension to another
    for filename in os.listdir(dir_src):
        if filename.endswith('.csv'):
            shutil.copy(dir_src + filename, dir_dst)

def ensure_dir(file_path): #ensure a dictory exists otherwise create it
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        print('New Directory Created',file_path)

####


In [8]:
ticker_data={}
date_data={}
data_trades={}
raw_data=  pd.read_csv('/home/ak/Documents/Data/GPDATA/garethpeters@unsw.edu.au-Jan2012_HMMProj-N145855118.csv.tar.gz', compression='gzip')
raw_data.columns = ['#RIC' if x=='garethpeters@unsw.edu.au-Jan2012_HMMProj-N145855118.csv' else x for x in raw_data.columns]

for i in range(no_files):
     print i
     #raw_data= comp_data #_raw_data(_path + _files[i])
     data_trades[i] =_trade_data(raw_data)
     data_trades[i]= _basic_columns(data_trades[i])
     data_trades[i].rename(columns={'Price': 'TradedPrice', 'Time[G]': 'TradedTime'}, inplace=True)
     data_trades[i]['ReturnTradedPrice'] =data_trades[i]['TradedPrice'].pct_change()
     data_trades[i]['Duration']= data_trades[i]['TradedTime'].apply(time_in_millis).diff().astype(float)
     data_trades[i]['logReturnTradedPrice']  = np.log(1 + data_trades[i]['ReturnTradedPrice'])
     #data_trades[i]= data_trades[i].ix[1:] #--check this works before you run it pls



#print raw_data.head(10)
print 'Did the First Bit'

ticker_list=data_trades[0]['#RIC'].unique()
for ticker_ in ticker_list:
    if not os.path.exists(_path+ticker_):
        os.makedirs(_path+ticker_)

date_list =data_trades[0]['Date[G]'].unique()
#print(ticker_list)
#
for j in range(no_files):
    for ticker_ in ticker_list:
        root=(_path + ticker_)
        ticker_data[ticker_] = data_trades[j][data_trades[j]['#RIC']==ticker_]
        #ticker_data[ticker_].to_csv(_path+ ticker_+ '.csv')
        for date_ in date_list:
            date_data[date_]=ticker_data[ticker_][ticker_data[ticker_]['Date[G]'] == date_]
            #date_data[date_]['Date[G]'].apply(_date_convert)
            date_data[date_].to_csv(root + '/' +date_ +'.csv')

            #date_data[date_].dropna()


print 'Finished. check your directory'
print date_data[date_list[0]].head(10)


/home/ak/Envs/DataAnalysis/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
Did the First Bit
Finished. check your directory
          #RIC      Date[G]       TradedTime  TradedPrice   Volume  \
36177510  GD.N  03-JAN-2012  14:31:38.813725        67.97  23699.0   
36177513  GD.N  03-JAN-2012  14:31:38.813762        67.95    100.0   
36177515  GD.N  03-JAN-2012  14:31:38.813762        67.95    100.0   
36177521  GD.N  03-JAN-2012  14:31:38.823482        67.95    100.0   
36177532  GD.N  03-JAN-2012  14:31:38.898718        68.00    100.0   
36177540  GD.N  03-JAN-2012  14:31:39.199525        67.95    100.0   
36177542  GD.N  03-JAN-2012  14:31:39.199525        67.95    100.0   
36177567  GD.N  03-JAN-2012  14:31:42.627170        67.98    300.0   
36177573  GD.N  03-JAN-2012  14:31:42.627170        67.97    200.0   
36177574  GD.N  03-JAN-2012  14:31:42.627170        67.98   1100.0   

          ReturnTradedPrice      Duration  logReturnTradedPrice  
36177510          -0.174320 -2.332181e+07             -0.191548  
36177513          -0.000294  3.700000e-02     

In [13]:
def _date_convert(_date):
    return datetime.date.strftime(datetime.datetime.strptime(_date, '%d-%b-%Y'), '%Y%m%d')

In [14]:
symbols=listdirs(_path)
for index, symbol in enumerate(symbols):
    print index, symbol

0 MS.N
1 HSBA.L
2 YHOO.O
3 LMT.N
4 GD.N
5 DBKGn.DE
6 JPM.N
7 C.N
8 GS.N
9 GOOG.OQ


In [43]:
number_=9
symbol_list=os.listdir(_path+symbols[number_])
df1=pd.read_csv(_path+str(symbols[number_])+'/'+symbol_list[0])
df2=df1.iloc[1:]
df2.to_csv(data_path+symbols[number_]+'/'+symbol_list[0])
symbols[number_]
target_path=os.path.join(data_path,symbols[number_])
target_path

'/home/ak/Documents/Data/GOOG.OQ'

for index, _ in enumerate(symbols):
    folder=os.listdir(_path+symbols[index])
    no_files=len(folder)
    for j in range((len(folder))):
        df=pd.read_csv(_path+symbols[index]+'/'+folder[j])
        _df=df.iloc[1:]
        _df.to_csv(data_path+symbols[index]+'/'+folder[j])
            
            
            

In [44]:
symbol_path= os.path.join(data_path,'GPDATA',symbols[number_])
symbol_path

'/home/ak/Documents/Data/GPDATA/GOOG.OQ'

In [45]:
_files =os.listdir(symbol_path)
# file_name=_files[1]
# os.path.splitext(os.path.split(file_name)[1])[0]
#_files
symbol_path
len(_files)

26

In [46]:
for index,date_string in enumerate(_files):
    print index
    new_date=_date_convert(os.path.splitext(os.path.split(date_string)[1])[0])
    df=pd.read_csv(symbol_path+'/'+_files[index])
    _df=df.iloc[1:]
    _df.to_csv(target_path+'/'+new_date+'.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [47]:
data_dir = os.getenv('FINANCE_DATA') 
#df = pd.read_csv(os.path.join(data_dir, 'YHOO.O', '20120119.csv'))

import fnmatch
list_models=fnmatch.filter(os.listdir(_models_path), str(ticker))

In [48]:
len(os.listdir(os.path.join(data_dir,str(symbols[number_]))))

27

In [49]:
len(os.listdir(target_path))

27